# Cryosphere model Comparison tool (CmCt) --- Calving

The CmCt Calving tool compares user uploaded ice sheet model to satellite calving data, derived from the   satellite missions. The CmCt reprojects modeled ice sheet mass change into calving space and performs a direct quantitative comparison, calculating model-observation discrepancies, therefore bringing the ice sheet model and calving data to the same spatial resolution for comparison.

## Calving dataset description

For comparison to the user's model, the tool uses the NASA GSFC GRACE mascon product, which can be downloaded from [here](https://earth.gsfc.nasa.gov/geo/data/grace-mascons/). Future developments will include adding the functionality to automatically check the product distribution website for an updated dataset.

## Input data requirements

The input ice sheet model needs to be provided as a netCDF file. The user may upload a single input file that includes multiple years; future enhancements to this tool will allow users to upload model ensembles.

There are several requirements for the comparison:

### `Lithk` variable

The CmCt Grace Mascon tool expects the uploaded model to contain thickness data (the `lithk` variable) for the comparison.

### Rectangular grid

At time of writing, models *must* be defined on a rectangular X-Y grid in the ISMIP6 standard projected polar-stereographic space. (Note, NOT a lat-lon grid!) The ISMIP6 standard projection is defined [here](https://theghub.org/groups/ismip6/wiki). 

### Date range

The NASA GSFC GRACE mascon dataset starts on 04/2002 and is updated ~quarterly. The user can select start and end dates within this span as part of the setup for the tool.

In [1]:
## Import modules
import os,sys
import numpy as np
import cftime 

# Add the directory containing 'cmct' to the Python path
# Navigate two levels up to reach main CmCt dir
cmct_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))

# Import utilities for this comparison
sys.path.insert(0,cmct_dir)
from cmct.time_utils import check_datarange
from cmct.calving import *
from cmct.gravimetry import *
from cmct.projection import *


### Finding the Dataset

In [2]:
# Ice sheet
loc = 'GIS' # 'GIS' or 'AIS'

# Set time range for comparison
start_date = '2006-01-01' 
end_date = '2014-12-31' 

# Set the observation data dir path
obs_filename = cmct_dir + '/data/calving/sftgif_GIS_JPL_ISSM_historical.nc'


# LEGACY Gravimetry data paths

# # Set the model data dir path
# if loc == "GIS":
#     # Greenland
#     nc_filename = cmct_dir + '/test/gris_dmdt_filt.nc'
# elif loc == "AIS":    
#     # Antartica
#     nc_filename = cmct_dir + '/test/ais_dmdt_grounded_filt.nc'

# # Set the output plot file names
# plot_filename = os.path.splitext(os.path.basename(nc_filename))[0] + '_calving_comp'
# output_plot_filename = cmct_dir + '/notebooks/Calving/'+ plot_filename + '.png'
# output_netcdf_filename = cmct_dir + '/notebooks/Calving/'+ plot_filename + '.nc'


In [3]:
# Check if observation file exist
if not os.path.exists(obs_filename):
    raise FileNotFoundError(f"Observation file not found: {obs_filename}")

# # Check if model file exist
# if not os.path.exists(nc_filename):
#     raise FileNotFoundError(f"Model file not found: {nc_filename}")


## Set polar sterographic projection definition
Currently, the CmCt expects the model data to be provided in a specific polar stereographic projection. This is hardcoded in the notebook. As a future enhancement, the projection can be obtained from the loaded model.

In [4]:
polar_stereographic = set_projection(loc)

## Process observation

In [5]:
print(obs_filename)
gsfc = load_gsfc_calving(obs_filename)
print(type(gsfc))
print(gsfc.print_info())  # Print the gsfc object to see its contents
print(vars(gsfc))  # Print the attributes of the gsfc object

/Users/aditya_pachpande/Documents/GitHub/CmCt/data/calving/sftgif_GIS_JPL_ISSM_historical.nc
<class 'cmct.calving.GSFCcalving'>
GSFC Calving Data:
Latitude: [[58.26977771 58.27849862 58.28716162 ... 57.80706733 57.79569481
  57.78426749]
 [58.31170704 58.32044184 58.32911864 ... 57.84826994 57.83687981
  57.8254348 ]
 [58.35364224 58.36239096 58.3710816  ... 57.88947658 57.87806879
  57.86660604]
 ...
 [81.48028045 81.51598629 81.55159474 ... 79.75134913 79.71222952
  79.67305699]
 [81.50900734 81.5448362  81.58056838 ... 79.77514171 79.73592955
  79.69666504]
 [81.53758312 81.57353519 81.60939128 ... 79.79878418 79.75947964
  79.72012333]]
Longitude: [[-56.7881833  -56.70858858 -56.62894799 ... -29.60445075 -29.52729455
  -29.45019583]
 [-56.80481266 -56.72511202 -56.64536532 ... -29.58316827 -29.50591599
  -29.42872141]
 [-56.82148834 -56.7416815  -56.66182843 ... -29.56182831 -29.48447972
  -29.40718906]
 ...
 [-96.14662566 -95.95142228 -95.75456533 ...  13.59483633  13.72844101
   

TypeError: unsupported operand type(s) for *: 'cftime._cftime.DatetimeNoLeap' and 'int'